Usaremos o GridSearch para tentar otimizar o modelo que obteve melhores resultados no dia 3

In [ ]:
# DIA 4 - OTIMIZAÇÃO E INTERPRETAÇÃO

# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import joblib  # para salvar o modelo treinado

# Carregar dados salvos no Dia 2
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv").values.ravel()
y_test = pd.read_csv("y_test.csv").values.ravel()

print("✅ Dados carregados com sucesso!")
print(f"Treino: {X_train.shape}, Teste: {X_test.shape}")


In [ ]:
# Definir modelo base e grade de hiperparâmetros 

modelo_base = GradientBoostingClassifier(random_state=42)

# Grade de parâmetros para busca
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

print("Grade de parâmetros definida.")


In [ ]:
# Otimização com GridSearchCV 
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=modelo_base,
    param_grid=param_grid,
    scoring='recall_weighted',  # métrica principal: recall
    cv=cv,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("\n✅ Busca concluída!")
print("Melhores parâmetros encontrados:")
print(grid_search.best_params_)


In [ ]:
# Avaliar o modelo otimizado

melhor_modelo = grid_search.best_estimator_

# Previsões no conjunto de teste
y_pred = melhor_modelo.predict(X_test)

# Calcular métricas
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted')
rec = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("=== Desempenho do modelo otimizado ===")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")

print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))


In [ ]:
# Importância das variáveis (feature importance)

importances = melhor_modelo.feature_importances_
features = X_train.columns

# Ordenar pela importância
indices = np.argsort(importances)[::-1]
sorted_features = features[indices]
sorted_importances = importances[indices]

# Exibir as 10 mais relevantes
top_n = 10
plt.figure(figsize=(10,6))
sns.barplot(x=sorted_importances[:top_n], y=sorted_features[:top_n], palette="viridis")
plt.title("Top 10 Variáveis Mais Importantes - Gradient Boosting")
plt.xlabel("Importância")
plt.ylabel("Variável")
plt.tight_layout()
plt.show()


In [ ]:
# Salvar o modelo otimizado

joblib.dump(melhor_modelo, "modelo_gradient_boosting_otimizado.pkl")
print("✅ Modelo otimizado salvo como 'modelo_gradient_boosting_otimizado.pkl'")


CONCLUSÃO: O GridSearch não obteve resultados que otimizassem o Gradient Boosting. Entretanto, este algoritmo continua sendo o melhor para a analise preditiva de evasão academica, tendo em vista os resultados e justificativas ressaltadas no notebook do dia 3.
Portanto, este trabalho é concluido provando a eficacia do Gradient Boosting para a previsão de evasão academica em relação aos outros modelos de aprendizado usados. Para além disso, todos os modelos obtiveram bom resultados em torno de 0.88 nas metricas de avaliação de resultados 